In [ ]:
!pip install langchain
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install sentence-transformers
!pip install ipywidgets

In [1]:
from ipywidgets import IntProgress
from IPython.display import display
from langchain.prompts import PromptTemplate
import io
from langchain.text_splitter import CharacterTextSplitter
import requests
from tqdm import tqdm

In [2]:
url = "http://127.0.0.1:5000/v1/chat/completions"
# url = "http://127.0.0.1:5000/v1/completions"
headers = {
    "Content-Type": "application/json"
}
history = []

In [3]:
f = io.open("n4764du.txt", mode="r", encoding="utf-8")
s = f.read()

In [4]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 250,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

In [5]:
texts = text_splitter.split_text(s)

Created a chunk of size 267, which is longer than the specified 250
Created a chunk of size 302, which is longer than the specified 250
Created a chunk of size 262, which is longer than the specified 250
Created a chunk of size 276, which is longer than the specified 250
Created a chunk of size 262, which is longer than the specified 250
Created a chunk of size 260, which is longer than the specified 250
Created a chunk of size 271, which is longer than the specified 250
Created a chunk of size 286, which is longer than the specified 250
Created a chunk of size 270, which is longer than the specified 250
Created a chunk of size 284, which is longer than the specified 250
Created a chunk of size 257, which is longer than the specified 250
Created a chunk of size 280, which is longer than the specified 250
Created a chunk of size 276, which is longer than the specified 250
Created a chunk of size 258, which is longer than the specified 250
Created a chunk of size 293, which is longer tha

In [6]:
len(texts)

6039

In [ ]:
template = """
### Instruction:
Translate Japanese to Chinese:
{question}

### Response:
中文：
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
template = """
Translate Japanese to tradational Chinese:
{question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [7]:
template = """
Translate Japanese to Chinese:
{question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [8]:
texts[7]

'真名があったお陰で、錬金術の師匠に引き取られ、錬金術師として身を立てることができた。\n錬金術のスキルは珍しいものではなかったし、ポーションは回復魔法にくらべて使い勝手が悪い。\n錬金術師は儲からない職業だったけれど、師匠が残してくれた魔の森の外れの小屋で、細々と暮らす位は出来た。\n豊かではなかったけれど、様々なポーションを作るのは面白かったし、人付き合いが苦手なマリエラにとって、静かな森での暮らしは性に合っていた。\nずっと続くと思っていたささやかな暮らしは、ある日突然終わりを迎えた。'

In [9]:
data = {
    "messages": [
      {
        "role": "user",
        "content": prompt.format(question=texts[7])
      }
    ],
    "mode": "instruct", #instruct
    "instruction_template": "Alpaca",
    "temperature": 0.7,
    "top_p": 0.9,
    # "stream": False,
    # "max_new_tokens": 4096,
    # "do_sample": True,
    # "typical_p": 1,
    # "repetition_penalty": 1.1,
    # "encoder_repetition_penalty": 1.0,
    # "top_k": 0,
    # "min_length": 0,
    # "no_repeat_ngram_size": 0,
    # "num_beams": 1,
    # "penalty_alpha": 0,
    # "length_penalty": 1,
    # "early_stopping": False,
    # "seed": -1,
    # "add_bos_token": True,
    # "custom_stopping_strings": [],
    # "truncation_length": 4096,
    # "ban_eos_token": False
}

url = "http://127.0.0.1:5000/v1/chat/completions"

response = requests.post(url, headers=headers, json=data, verify=False)

response.json()

print(response.json()['choices'][0]['message']['content'])

由于有了真名，我被炼金术的师父收养，得以成为一名炼金术师，养活自己。虽然炼金术的技能并不罕见，而且药水相比于恢复魔法，使用起来并不方便。炼金术师并不是一个赚钱的职业，但师父留下的位于魔之森边缘的小屋里，我能够细水长流地生活下去。虽然生活并不富裕，但制作各种药水很有趣，对于不擅长与人交往的玛丽埃拉来说，在宁静的森林中生活是她的天性。原本以为会持续下去的平淡生活，在某一天突然迎来了终结。


In [11]:
result=[]
p2 = tqdm(total=len(texts), desc="Running", dynamic_ncols=True, position=0, leave=True)
# p2 = IntProgress(max=len(texts))
# p2.description = 'Running'
# display(p2)
x = 0
with open(r'n4764du-qwen15.txt', 'a', encoding='UTF-8') as fp:
    for i in texts:
        #p2.value += 1
        p2.update(1)
        x += 1
        if x > 0:
            data = {
                    "messages": [
                          {
                            "role": "user",
                            "content": prompt.format(question=i)
                          }
                                ],
                    "mode": "instruct", #instruct
                    "instruction_template": "Alpaca",
                    "temperature": 0.7,
                    "top_p": 0.9,
                    }
            response = requests.post(url, headers=headers, json=data, verify=False)
            rs = response.json()['choices'][0]['message']['content']
            fp.write('\n'+ rs)
            fp.flush()
    fp.close()

Running:   0%|                                                                   | 8/6039 [26:52<360:35:39, 215.24s/it]

KeyboardInterrupt: 

In [ ]:
from tqdm import tqdm